In [1]:
#imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#sklearn stuff
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA

from math import sqrt
%matplotlib inline

In [2]:
# Import the Dataset
data = pd.read_csv('Capstone_Dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 11 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Date                                                                              365 non-null    object 
 1   WTI Price (Dollars Per Barrel)                                                    365 non-null    float64
 2   U.S. Imports of Crude Oil (Thousand Barrels)                                      365 non-null    int64  
 3   U.S. Exports of Crude Oil (Thousand Barrels)                                      365 non-null    int64  
 4   U.S. Crude Oil Rotary Rigs
in Operation (Count)                                   365 non-null    int64  
 5   U.S. Field Production of Crude Oil (Thousand Barrels per Day)                     365 non-null    int64  
 6   Ea

In [3]:
data.head(10)

,Date,WTI Price (Dollars Per Barrel),U.S. Imports of Crude Oil (Thousand Barrels),U.S. Exports of Crude Oil (Thousand Barrels),U.S. Crude Oil Rotary Rigs\nin Operation (Count),U.S. Field Production of Crude Oil (Thousand Barrels per Day),East Coast (PADD 1) Field Production of Crude Oil (Thousand Barrels per Day),Midwest (PADD 2) Field Production of Crude Oil (Thousand Barrels per Day),Gulf Coast (PADD 3) Field Production of Crude Oil (Thousand Barrels per Day),Rocky Mountain (PADD 4) Field Production of Crude Oil (Thousand Barrels per Day),West Coast (PADD 5) Field Production of Crude Oil (Thousand Barrels per Day)
0,Jan-1990,22.86,192572,4084,514,7546,35,785,3374,505,2847
1,Feb-1990,22.11,165046,2843,492,7497,35,744,3401,508,2809
2,Mar-1990,20.39,189625,4104,478,7433,31,745,3348,508,2801
3,Apr-1990,18.43,174393,3338,484,7407,32,736,3354,502,2783
4,May-1990,18.20,200067,3474,493,7328,31,737,3310,506,2744
5,Jun-1990,16.70,192704,2639,498,7106,29,723,3280,499,2575
6,Jul-1990,18.45,212514,2762,497,7173,27,727,3282,488,2649
7,Aug-1990,27.31,200000,1990,506,7287,28,762,3316,492,2690
8,Sep-1990,33.51,169907,2050,566,7224,27,732,3306,492,2667
9,Oct-1990,36.04,159080,3233,588,7542,28,763,3388,498,2864


In [5]:
# Rename columns
data['Oil_Price'] = data['WTI Price (Dollars Per Barrel)']
data['Oil_Imports'] = data['U.S. Imports of Crude Oil (Thousand Barrels)']
data['Oil_Exports'] = data['U.S. Exports of Crude Oil (Thousand Barrels)']
data['Rig_Count'] = data['U.S. Crude Oil Rotary Rigs\nin Operation (Count)']
data['U.S._Oil_Production'] = data['U.S. Field Production of Crude Oil (Thousand Barrels per Day)']
data['EastCoast_Oil_Production'] = data['East Coast (PADD 1) Field Production of Crude Oil (Thousand Barrels per Day)']
data['Midwest_Oil_Production'] = data['Midwest (PADD 2) Field Production of Crude Oil (Thousand Barrels per Day)']
data['GulfCoast_Oil_Production'] = data['Gulf Coast (PADD 3) Field Production of Crude Oil (Thousand Barrels per Day)']
data['RockyMountain_Oil_Production'] = data['Rocky Mountain (PADD 4) Field Production of Crude Oil (Thousand Barrels per Day)']
data['WestCoast_Oil_Production'] = data['West Coast (PADD 5) Field Production of Crude Oil (Thousand Barrels per Day)']

# Drop unneeded columns
data = data.drop(['Date', 'WTI Price (Dollars Per Barrel)', 'U.S. Imports of Crude Oil (Thousand Barrels)',
                 'U.S. Exports of Crude Oil (Thousand Barrels)', 'U.S. Crude Oil Rotary Rigs\nin Operation (Count)',
                 'U.S. Field Production of Crude Oil (Thousand Barrels per Day)',
                 'East Coast (PADD 1) Field Production of Crude Oil (Thousand Barrels per Day)',
                 'Midwest (PADD 2) Field Production of Crude Oil (Thousand Barrels per Day)',
                 'Gulf Coast (PADD 3) Field Production of Crude Oil (Thousand Barrels per Day)',
                 'Rocky Mountain (PADD 4) Field Production of Crude Oil (Thousand Barrels per Day)',
                 'West Coast (PADD 5) Field Production of Crude Oil (Thousand Barrels per Day)'], axis=1)

data.head()

,Oil_Price,Oil_Imports,Oil_Exports,Rig_Count,U.S._Oil_Production,EastCoast_Oil_Production,Midwest_Oil_Production,GulfCoast_Oil_Production,RockyMountain_Oil_Production,WestCoast_Oil_Production
0,22.86,192572,4084,514,7546,35,785,3374,505,2847
1,22.11,165046,2843,492,7497,35,744,3401,508,2809
2,20.39,189625,4104,478,7433,31,745,3348,508,2801
3,18.43,174393,3338,484,7407,32,736,3354,502,2783
4,18.20,200067,3474,493,7328,31,737,3310,506,2744


In [6]:
# Check the data types
data.dtypes

Oil_Price                       float64
Oil_Imports                       int64
Oil_Exports                       int64
Rig_Count                         int64
U.S._Oil_Production               int64
EastCoast_Oil_Production          int64
Midwest_Oil_Production            int64
GulfCoast_Oil_Production          int64
RockyMountain_Oil_Production      int64
WestCoast_Oil_Production          int64
dtype: object

In [7]:
# Drop duplicate rows
data = data.drop_duplicates()

# Check for nulls
nulls = data.isnull().sum()
nulls

Oil_Price                       0
Oil_Imports                     0
Oil_Exports                     0
Rig_Count                       0
U.S._Oil_Production             0
EastCoast_Oil_Production        0
Midwest_Oil_Production          0
GulfCoast_Oil_Production        0
RockyMountain_Oil_Production    0
WestCoast_Oil_Production        0
dtype: int64

In [8]:
# View the makeup of the data
data.describe()

,Oil_Price,Oil_Imports,Oil_Exports,Rig_Count,U.S._Oil_Production,EastCoast_Oil_Production,Midwest_Oil_Production,GulfCoast_Oil_Production,RockyMountain_Oil_Production,WestCoast_Oil_Production
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,47.586082,250595.868493,10582.882192,503.287671,6938.547945,30.693151,888.528767,3879.709589,463.471233,1676.153425
std,29.108127,42389.848480,22408.439895,378.852543,1885.005407,14.760695,567.528309,1417.490226,185.486167,601.029107
min,11.350000,142930.000000,52.000000,108.000000,3974.000000,14.000000,412.000000,1664.000000,281.000000,803.000000
25%,21.280000,221130.000000,975.000000,244.000000,5607.000000,21.000000,473.000000,3135.000000,349.000000,1134.000000
50%,40.750000,248909.000000,2241.000000,358.000000,6442.000000,26.000000,596.000000,3255.000000,383.000000,1597.000000
75%,67.870000,284528.000000,4646.000000,668.000000,7433.000000,31.000000,1248.000000,4047.000000,502.000000,2212.000000
max,133.880000,327476.000000,113731.000000,1596.000000,12860.000000,83.000000,2332.000000,8660.000000,1021.000000,2878.000000


The price of oil ranges from 11.35 to 133.88 with a mean of 47.58 and standard deviation of 29.1. The Gulf Coast Region has the highest production of all the Regions with a mean of 3879.7 Thousand Barrels Per Day and the East Coast Region has the lowest production with a mean of 30.69 Thousand Barrels Per Day.

In [15]:
# Check number of distinct values for Oil Price
data['Oil_Price'].value_counts().shape

(348,)

In [9]:
# Check the correlations of the variables
data.corr()

,Oil_Price,Oil_Imports,Oil_Exports,Rig_Count,U.S._Oil_Production,EastCoast_Oil_Production,Midwest_Oil_Production,GulfCoast_Oil_Production,RockyMountain_Oil_Production,WestCoast_Oil_Production
Oil_Price,1.000000,0.300876,0.081131,0.645806,0.019949,0.138084,0.337257,0.169090,0.223246,-0.726984
Oil_Imports,0.300876,1.000000,-0.325868,-0.290850,-0.590451,-0.494174,-0.408121,-0.390281,-0.498985,-0.379709
Oil_Exports,0.081131,-0.325868,1.000000,0.259742,0.867794,0.811246,0.770525,0.911947,0.849272,-0.438754
Rig_Count,0.645806,-0.290850,0.259742,1.000000,0.465431,0.533553,0.667703,0.474204,0.544584,-0.470431
U.S._Oil_Production,0.019949,-0.590451,0.867794,0.465431,1.000000,0.953711,0.920202,0.963175,0.960199,-0.324072
EastCoast_Oil_Production,0.138084,-0.494174,0.811246,0.533553,0.953711,1.000000,0.939823,0.939674,0.946852,-0.429356
Midwest_Oil_Production,0.337257,-0.408121,0.770525,0.667703,0.920202,0.939823,1.000000,0.944143,0.962669,-0.605206
GulfCoast_Oil_Production,0.169090,-0.390281,0.911947,0.474204,0.963175,0.939674,0.944143,1.000000,0.951945,-0.546096
RockyMountain_Oil_Production,0.223246,-0.498985,0.849272,0.544584,0.960199,0.946852,0.962669,0.951945,1.000000,-0.474619
WestCoast_Oil_Production,-0.726984,-0.379709,-0.438754,-0.470431,-0.324072,-0.429356,-0.605206,-0.546096,-0.474619,1.000000


The variables West Coast Oil Production and Rig Count have the highest correlation with Oil Price in the dataset. U.S. Oil Production and Oil Exports have the lowest correlation with Oil Price in the dataset.

All datatypes are numeric values, there are no duplicate rows or null values, the column 'Date' has been removed and all column names have been changed.

In [10]:
# Save the cleaned dataset in a csv file
data.to_csv('Capstone_Dataset_Cleaned.csv', index=False)